In [1]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)


In [2]:
%cd data
os.getcwd()

d:\Documents\Github\projs\home-credit-default-risk\data


'd:\\Documents\\Github\\projs\\home-credit-default-risk\\data'

In [3]:
def show_cat_col_values(df):
    cats = df.select_dtypes(include = ['object'])

    for col in cats:
        ratios = []
        values = pd.DataFrame(df[col].value_counts())
        for value in values[col].values:
            total = np.sum(values[col].values)
            ratios.append(value/total)

        values['Ratio'] = ratios

        print(values.sort_values(by = 'Ratio', ascending = False))
        print("\n")

def show_missing_values(df):
    na_dict = {}
    for col in df.columns:
        na = df[col].isna().sum()
        total = len(df[col])

        na_dict[col] = f"{round(100*na/total,2)}%"

    print(pd.Series(na_dict).sort_values(ascending = False))

def remove_cat_col_values(df, threshold):
    """
    If one single value dominates the entire column, then we remove the column entirely, as there is too little variation in the column values
    """
    assert 0 <= threshold <= 1, "threshold must be between 0 and 1"
    cats = df.select_dtypes(include = ['object'])
    for col in cats:
        print(col)
        values = pd.DataFrame(df[col].value_counts())
        for value in values[col].values:
            total = np.sum(values[col].values)
            if value/total >= threshold:
                df = df.drop([col], axis = 1)
    return df


def remove_missing_values(df, threshold):
    assert 0 <= threshold <= 1, "threshold must be between 0 and 1"

    for col in df.columns:
        na = df[col].isna().sum()
        total = len(df[col])

        if na/total > threshold:
            df = df.drop([col], axis = 1)

    return df

def remove_bad_rows(df, col, value):
    rows = df.loc[df[col] == value]
    
    df = df.drop(rows.index, axis = 0)
    
    return df

def show_ratio(df, target, col):
    dict_ = {}
    for value in set(df[col].values):
        
        target_1 = df.loc[(df[target] == 1) & (df[col] == value)]
        target_0 = df.loc[(df[target] == 0) & (df[col] == value)]

        if len(target_1) == len(target_0) == 0:
            dict_[value] = -1
        elif len(target_0) == 0:
            dict_[value] = -2
        else:
            dict_[value] = len(target_1)/len(target_0)
    
    series = pd.Series(dict_).sort_values(ascending = False)
    series = series.replace({-1: "no values found", -2: f"{value} has all target values of 1"})
    print(col)
    print(series)
    print("\n")

def show_ratio_by_target(df, target, cols = None):
    """
    This can take either a list, string, or all of the object dataframes in the dataframe
    """
    if isinstance(cols, str):
        show_ratio(df, target, cols)

    elif isinstance(cols, list):
        for col in cols:
            if col != target:
                show_ratio(df, target, col)

    elif isinstance(df, pd.DataFrame):
        for col in df.select_dtypes('object'):
            if col != target:
                show_ratio(df, target, col)

    else:
        return "Invalid type for parameter col"


def show_unique_keys(df):
    for col in df.columns:
        print(col, df[col].is_unique)

def get_cat_agg(df, col, type):
    cat_agg = {}
    for i in range(len(df.columns)):
        if col in df.columns[i]:
            cat_agg[df.columns[i]] = type
    
    return cat_agg


Bureau

In [4]:
raw_bureau = pd.read_csv(os.getcwd()+"\\bureau.csv")

Here we can see that the unique key is SK_ID_BUREAU, which we can use to combine data from the bureau_balance.csv file

In [5]:
show_unique_keys(raw_bureau)

SK_ID_CURR False
SK_ID_BUREAU True
CREDIT_ACTIVE False
CREDIT_CURRENCY False
DAYS_CREDIT False
CREDIT_DAY_OVERDUE False
DAYS_CREDIT_ENDDATE False
DAYS_ENDDATE_FACT False
AMT_CREDIT_MAX_OVERDUE False
CNT_CREDIT_PROLONG False
AMT_CREDIT_SUM False
AMT_CREDIT_SUM_DEBT False
AMT_CREDIT_SUM_LIMIT False
AMT_CREDIT_SUM_OVERDUE False
CREDIT_TYPE False
DAYS_CREDIT_UPDATE False
AMT_ANNUITY False


In [6]:
raw_bureau_dropped = raw_bureau.drop(['SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY'], axis = 1)

Dataset Analysis for Bureau

In [7]:
raw_bureau_dropped.head()

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [8]:
show_missing_values(raw_bureau_dropped)

AMT_ANNUITY               71.47%
AMT_CREDIT_MAX_OVERDUE    65.51%
DAYS_CREDIT_ENDDATE        6.15%
DAYS_ENDDATE_FACT         36.92%
AMT_CREDIT_SUM_LIMIT      34.48%
AMT_CREDIT_SUM_DEBT       15.01%
SK_ID_CURR                  0.0%
DAYS_CREDIT                 0.0%
CREDIT_DAY_OVERDUE          0.0%
CNT_CREDIT_PROLONG          0.0%
AMT_CREDIT_SUM              0.0%
AMT_CREDIT_SUM_OVERDUE      0.0%
CREDIT_TYPE                 0.0%
DAYS_CREDIT_UPDATE          0.0%
dtype: object


In [9]:
raw_bureau_dropped.dtypes

SK_ID_CURR                  int64
DAYS_CREDIT                 int64
CREDIT_DAY_OVERDUE          int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG          int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE                object
DAYS_CREDIT_UPDATE          int64
AMT_ANNUITY               float64
dtype: object

In [10]:
show_cat_col_values(raw_bureau_dropped)

                                              CREDIT_TYPE         Ratio
Consumer credit                                   1251615  7.291975e-01
Credit card                                        402195  2.343209e-01
Car loan                                            27690  1.613234e-02
Mortgage                                            18391  1.071469e-02
Microloan                                           12413  7.231879e-03
Loan for business development                        1975  1.150645e-03
Another type of loan                                 1017  5.925096e-04
Unknown type of loan                                  555  3.233459e-04
Loan for working capital replenishment                469  2.732419e-04
Cash loan (non-earmarked)                              56  3.262590e-05
Real estate loan                                       27  1.573034e-05
Loan for the purchase of equipment                     19  1.106950e-05
Loan for purchase of shares (margin lending)            4  2.330

Feature Engineering for Bureau

In [11]:
raw_bureau_dropped['DAYS_CREDIT'].replace(365243, np.nan)
raw_bureau_dropped['DAYS_CREDIT_ENDDATE'].replace(365243, np.nan)
raw_bureau_dropped['DAYS_ENDDATE_FACT'].replace(365243, np.nan)
raw_bureau_dropped['DAYS_CREDIT_UPDATE'].replace(365243, np.nan)

0          -131
1           -20
2           -16
3           -16
4           -21
           ... 
1716423     -19
1716424   -2493
1716425    -967
1716426   -1508
1716427    -387
Name: DAYS_CREDIT_UPDATE, Length: 1716428, dtype: int64

In [12]:
raw_bureau_cat = pd.get_dummies(raw_bureau_dropped)

Aggregating Data by SK_ID_CURR in the Bureau Dataset

In [13]:
bureau_agg_num = {
    "DAYS_CREDIT":["mean"],
    "CREDIT_DAY_OVERDUE":["mean"],
    "DAYS_CREDIT_ENDDATE":["min"],
    "DAYS_ENDDATE_FACT":["max"],
    "AMT_CREDIT_MAX_OVERDUE":["max"],
    "CNT_CREDIT_PROLONG":["max"],
    "AMT_CREDIT_SUM":["min","mean", "max", "var"],
    "AMT_CREDIT_SUM_DEBT":["mean"],
    "AMT_CREDIT_SUM_LIMIT":["max"],
    "AMT_CREDIT_SUM_OVERDUE":["max"],
    "DAYS_CREDIT_UPDATE":["max"],
    "AMT_ANNUITY":["min","mean", "max", "var"],
}

bureau_agg_cat = get_cat_agg(raw_bureau_cat, "CREDIT_TYPE", 'sum')

bureau_agg_num.update(bureau_agg_cat)

bureau_agg_num

{'DAYS_CREDIT': ['mean'],
 'CREDIT_DAY_OVERDUE': ['mean'],
 'DAYS_CREDIT_ENDDATE': ['min'],
 'DAYS_ENDDATE_FACT': ['max'],
 'AMT_CREDIT_MAX_OVERDUE': ['max'],
 'CNT_CREDIT_PROLONG': ['max'],
 'AMT_CREDIT_SUM': ['min', 'mean', 'max', 'var'],
 'AMT_CREDIT_SUM_DEBT': ['mean'],
 'AMT_CREDIT_SUM_LIMIT': ['max'],
 'AMT_CREDIT_SUM_OVERDUE': ['max'],
 'DAYS_CREDIT_UPDATE': ['max'],
 'AMT_ANNUITY': ['min', 'mean', 'max', 'var'],
 'CREDIT_TYPE_Another type of loan': 'sum',
 'CREDIT_TYPE_Car loan': 'sum',
 'CREDIT_TYPE_Cash loan (non-earmarked)': 'sum',
 'CREDIT_TYPE_Consumer credit': 'sum',
 'CREDIT_TYPE_Credit card': 'sum',
 'CREDIT_TYPE_Interbank credit': 'sum',
 'CREDIT_TYPE_Loan for business development': 'sum',
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)': 'sum',
 'CREDIT_TYPE_Loan for the purchase of equipment': 'sum',
 'CREDIT_TYPE_Loan for working capital replenishment': 'sum',
 'CREDIT_TYPE_Microloan': 'sum',
 'CREDIT_TYPE_Mobile operator loan': 'sum',
 'CREDIT_TYPE_Mortg

In [14]:
bureau_agg = raw_bureau_cat.groupby('SK_ID_CURR').agg(bureau_agg_num)

bureau_agg.columns = pd.Index(['bureau_' + col[0] + "_" + col[1] for col in bureau_agg.columns.tolist() if col[0] != 'SK_ID_CURR'])

bureau_agg = bureau_agg.reset_index()

In [15]:
bureau_agg.head()

,SK_ID_CURR,bureau_DAYS_CREDIT_mean,bureau_CREDIT_DAY_OVERDUE_mean,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_max,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_CNT_CREDIT_PROLONG_max,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_var,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_LIMIT_max,bureau_AMT_CREDIT_SUM_OVERDUE_max,bureau_DAYS_CREDIT_UPDATE_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_var,bureau_CREDIT_TYPE_Another type of loan_sum,bureau_CREDIT_TYPE_Car loan_sum,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_sum,bureau_CREDIT_TYPE_Consumer credit_sum,bureau_CREDIT_TYPE_Credit card_sum,bureau_CREDIT_TYPE_Interbank credit_sum,bureau_CREDIT_TYPE_Loan for business development_sum,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,bureau_CREDIT_TYPE_Loan for the purchase of equipment_sum,bureau_CREDIT_TYPE_Loan for working capital replenishment_sum,bureau_CREDIT_TYPE_Microloan_sum,bureau_CREDIT_TYPE_Mobile operator loan_sum,bureau_CREDIT_TYPE_Mortgage_sum,bureau_CREDIT_TYPE_Real estate loan_sum,bureau_CREDIT_TYPE_Unknown type of loan_sum
0,100001,-735.000000,0.0,-1329.0,-544.0,NaN,0,85500.0,207623.571429,378000.0,1.501717e+10,85240.928571,0.000,0.0,-6,0.0,3545.357143,10822.5,2.304583e+07,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0
1,100002,-874.000000,0.0,-1072.0,-36.0,5043.645,0,0.0,108131.945625,450000.0,2.133807e+10,49156.200000,31988.565,0.0,-7,0.0,0.000000,0.0,0.000000e+00,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0
2,100003,-1400.750000,0.0,-2434.0,-540.0,0.000,0,22248.0,254350.125000,810000.0,1.385846e+11,0.000000,810000.000,0.0,-43,NaN,NaN,NaN,NaN,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0
3,100004,-867.000000,0.0,-595.0,-382.0,0.000,0,94500.0,94518.900000,94537.8,7.144200e+02,0.000000,0.000,0.0,-382,NaN,NaN,NaN,NaN,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
4,100005,-190.666667,0.0,-128.0,-123.0,0.000,0,29826.0,219042.000000,568800.0,9.195354e+10,189469.500000,0.000,0.0,-11,0.0,1420.500000,4261.5,6.053461e+06,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0


In [16]:
print(len(bureau_agg))

305811


Bureau Balance

In [17]:
raw_bureau_balance = pd.read_csv(os.getcwd()+"\\bureau_balance.csv")

In [18]:
show_unique_keys(raw_bureau_balance)

SK_ID_BUREAU False
MONTHS_BALANCE False
STATUS False


Here we deem the bureau_balance to not be of any interest for our model, we will omit this dataset

POS Cash Balance

In [19]:
raw_cash_balance = pd.read_csv(os.getcwd()+"\\POS_CASH_balance.csv")

In [20]:
show_unique_keys(raw_cash_balance)

SK_ID_PREV False
SK_ID_CURR False
MONTHS_BALANCE False
CNT_INSTALMENT False
CNT_INSTALMENT_FUTURE False
NAME_CONTRACT_STATUS False
SK_DPD False
SK_DPD_DEF False


Data Analysis for POS Cash Balance

In [21]:
raw_cash_balance.dtypes

SK_ID_PREV                 int64
SK_ID_CURR                 int64
MONTHS_BALANCE             int64
CNT_INSTALMENT           float64
CNT_INSTALMENT_FUTURE    float64
NAME_CONTRACT_STATUS      object
SK_DPD                     int64
SK_DPD_DEF                 int64
dtype: object

In [22]:
show_missing_values(raw_cash_balance)

CNT_INSTALMENT           0.26%
CNT_INSTALMENT_FUTURE    0.26%
SK_ID_PREV                0.0%
SK_ID_CURR                0.0%
MONTHS_BALANCE            0.0%
NAME_CONTRACT_STATUS      0.0%
SK_DPD                    0.0%
SK_DPD_DEF                0.0%
dtype: object


In [23]:
show_cat_col_values(raw_cash_balance)

                       NAME_CONTRACT_STATUS         Ratio
Active                              9151119  9.149876e-01
Completed                            744883  7.447819e-02
Signed                                87260  8.724815e-03
Demand                                 7065  7.064041e-04
Returned to the store                  5461  5.460258e-04
Approved                               4917  4.916332e-04
Amortized debt                          636  6.359136e-05
Canceled                                 15  1.499796e-06
XNA                                       2  1.999728e-07




In [24]:
raw_cash_balance_dropped = raw_cash_balance.drop(['SK_ID_PREV', 'MONTHS_BALANCE', 'NAME_CONTRACT_STATUS'], axis = 1)

In [25]:
raw_cash_balance_dropped = raw_cash_balance_dropped.replace('XNA', np.nan)

Data Aggregation for POS Cash Balance

In [26]:
cash_balance_agg_num = {
"CNT_INSTALMENT":["min","mean", "max"],
"CNT_INSTALMENT_FUTURE":["min","mean", "max"],
"SK_DPD":["min","mean", "max"],
"SK_DPD_DEF":["min","mean", "max"],
}

In [27]:
cash_balance_agg = raw_cash_balance_dropped.groupby('SK_ID_CURR').agg(cash_balance_agg_num)

cash_balance_agg.columns = pd.Index(['cash_balance_' + col[0] + "_" + col[1] for col in cash_balance_agg.columns.tolist() if col[0] != 'SK_ID_CURR'])

cash_balance_agg = cash_balance_agg.reset_index()

In [28]:
cash_balance_agg.head()

,SK_ID_CURR,cash_balance_CNT_INSTALMENT_min,cash_balance_CNT_INSTALMENT_mean,cash_balance_CNT_INSTALMENT_max,cash_balance_CNT_INSTALMENT_FUTURE_min,cash_balance_CNT_INSTALMENT_FUTURE_mean,cash_balance_CNT_INSTALMENT_FUTURE_max,cash_balance_SK_DPD_min,cash_balance_SK_DPD_mean,cash_balance_SK_DPD_max,cash_balance_SK_DPD_DEF_min,cash_balance_SK_DPD_DEF_mean,cash_balance_SK_DPD_DEF_max
0,100001,4.0,4.000000,4.0,0.0,1.444444,4.0,0,0.777778,7,0,0.777778,7
1,100002,24.0,24.000000,24.0,6.0,15.000000,24.0,0,0.000000,0,0,0.000000,0
2,100003,6.0,10.107143,12.0,0.0,5.785714,12.0,0,0.000000,0,0,0.000000,0
3,100004,3.0,3.750000,4.0,0.0,2.250000,4.0,0,0.000000,0,0,0.000000,0
4,100005,9.0,11.700000,12.0,0.0,7.200000,12.0,0,0.000000,0,0,0.000000,0


In [29]:
len(cash_balance_agg)

337252

Credit Card Balance

In [30]:
raw_credit_card_balance = pd.read_csv(os.getcwd()+"\\credit_card_balance.csv")

Data Analysis for Credit Card Balance

In [31]:
raw_credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [32]:
show_missing_values(raw_credit_card_balance)

CNT_INSTALMENT_MATURE_CUM      7.95%
AMT_INST_MIN_REGULARITY        7.95%
AMT_PAYMENT_CURRENT            20.0%
AMT_DRAWINGS_ATM_CURRENT      19.52%
CNT_DRAWINGS_POS_CURRENT      19.52%
AMT_DRAWINGS_OTHER_CURRENT    19.52%
AMT_DRAWINGS_POS_CURRENT      19.52%
CNT_DRAWINGS_OTHER_CURRENT    19.52%
CNT_DRAWINGS_ATM_CURRENT      19.52%
SK_ID_PREV                      0.0%
AMT_TOTAL_RECEIVABLE            0.0%
SK_DPD                          0.0%
NAME_CONTRACT_STATUS            0.0%
CNT_DRAWINGS_CURRENT            0.0%
AMT_PAYMENT_TOTAL_CURRENT       0.0%
AMT_RECIVABLE                   0.0%
AMT_RECEIVABLE_PRINCIPAL        0.0%
SK_ID_CURR                      0.0%
AMT_DRAWINGS_CURRENT            0.0%
AMT_CREDIT_LIMIT_ACTUAL         0.0%
AMT_BALANCE                     0.0%
MONTHS_BALANCE                  0.0%
SK_DPD_DEF                      0.0%
dtype: object


In [33]:
show_cat_col_values(raw_credit_card_balance)

               NAME_CONTRACT_STATUS     Ratio
Active                      3698436  0.963056
Completed                    128918  0.033570
Signed                        11058  0.002879
Demand                         1365  0.000355
Sent proposal                   513  0.000134
Refused                          17  0.000004
Approved                          5  0.000001




In [34]:
raw_credit_card_balance_dropped = raw_credit_card_balance.drop(['SK_ID_PREV', 'NAME_CONTRACT_STATUS'], axis = 1)

In [35]:
raw_credit_card_balance_dropped.dtypes

SK_ID_CURR                      int64
MONTHS_BALANCE                  int64
AMT_BALANCE                   float64
AMT_CREDIT_LIMIT_ACTUAL         int64
AMT_DRAWINGS_ATM_CURRENT      float64
AMT_DRAWINGS_CURRENT          float64
AMT_DRAWINGS_OTHER_CURRENT    float64
AMT_DRAWINGS_POS_CURRENT      float64
AMT_INST_MIN_REGULARITY       float64
AMT_PAYMENT_CURRENT           float64
AMT_PAYMENT_TOTAL_CURRENT     float64
AMT_RECEIVABLE_PRINCIPAL      float64
AMT_RECIVABLE                 float64
AMT_TOTAL_RECEIVABLE          float64
CNT_DRAWINGS_ATM_CURRENT      float64
CNT_DRAWINGS_CURRENT            int64
CNT_DRAWINGS_OTHER_CURRENT    float64
CNT_DRAWINGS_POS_CURRENT      float64
CNT_INSTALMENT_MATURE_CUM     float64
SK_DPD                          int64
SK_DPD_DEF                      int64
dtype: object

Feature Engineering for Credit Card Balance

In [36]:
raw_credit_card_balance_dropped['PNT_BALANCE_DRAWINGS_ATM'] = raw_credit_card_balance_dropped['AMT_DRAWINGS_ATM_CURRENT']/raw_credit_card_balance_dropped['AMT_BALANCE']
raw_credit_card_balance_dropped['PNT_BALANCE_DRAWINGS_CURRENT'] = raw_credit_card_balance_dropped['AMT_DRAWINGS_ATM_CURRENT']/raw_credit_card_balance_dropped['AMT_BALANCE']
raw_credit_card_balance_dropped['AVG_DRAWINGS_ATM_CURRENT'] = raw_credit_card_balance_dropped['AMT_DRAWINGS_CURRENT']/raw_credit_card_balance_dropped['CNT_DRAWINGS_ATM_CURRENT']
raw_credit_card_balance_dropped['AVG_DRAWINGS_CURRENT'] = raw_credit_card_balance_dropped['AMT_DRAWINGS_CURRENT']/raw_credit_card_balance_dropped['CNT_DRAWINGS_CURRENT']
raw_credit_card_balance_dropped['AVG_DRAWINGS_OTHER_CURRENT'] = raw_credit_card_balance_dropped['AMT_DRAWINGS_OTHER_CURRENT']/raw_credit_card_balance_dropped['CNT_DRAWINGS_OTHER_CURRENT']
raw_credit_card_balance_dropped['AVG_DRAWINGS_POS_CURRENT'] = raw_credit_card_balance_dropped['AMT_DRAWINGS_POS_CURRENT']/raw_credit_card_balance_dropped['CNT_DRAWINGS_POS_CURRENT']

In [37]:
credit_card_balance_dropped_agg_num = {
"MONTHS_BALANCE":['max', 'mean', 'sum', 'median', 'std'],
"AMT_BALANCE": ['max', 'mean', 'sum', 'median', 'std'],
"AMT_CREDIT_LIMIT_ACTUAL": ['max', 'mean', 'sum', 'median', 'std'],
"AMT_DRAWINGS_ATM_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AMT_DRAWINGS_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AMT_DRAWINGS_OTHER_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AMT_DRAWINGS_POS_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AMT_INST_MIN_REGULARITY":['max', 'mean', 'sum', 'median', 'std'],
"AMT_PAYMENT_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AMT_PAYMENT_TOTAL_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AMT_RECEIVABLE_PRINCIPAL":['max', 'mean', 'sum', 'median', 'std'],
"AMT_RECIVABLE":['max', 'mean', 'sum', 'median', 'std'],
"AMT_TOTAL_RECEIVABLE":['max', 'mean', 'sum', 'median', 'std'],
"CNT_DRAWINGS_ATM_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"CNT_DRAWINGS_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"CNT_DRAWINGS_OTHER_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"CNT_DRAWINGS_POS_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"CNT_INSTALMENT_MATURE_CUM":['max', 'mean', 'sum', 'median', 'std'],
"SK_DPD":['max', 'mean', 'sum', 'median', 'std'],
"SK_DPD_DEF":['max', 'mean', 'sum', 'median', 'std'],
"PNT_BALANCE_DRAWINGS_ATM":['max', 'mean', 'sum', 'median', 'std'],
"PNT_BALANCE_DRAWINGS_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AVG_DRAWINGS_ATM_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AVG_DRAWINGS_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AVG_DRAWINGS_OTHER_CURRENT":['max', 'mean', 'sum', 'median', 'std'],
"AVG_DRAWINGS_POS_CURRENT":['max', 'mean', 'sum', 'median', 'std'],

}

In [38]:
credit_card_agg = raw_credit_card_balance_dropped.groupby('SK_ID_CURR').agg(credit_card_balance_dropped_agg_num)

credit_card_agg.columns = pd.Index(['credit_card_' + col[0] + "_" + col[1] for col in credit_card_agg.columns.tolist() if col[0] != 'SK_ID_CURR'])

credit_card_agg = credit_card_agg.reset_index()

In [39]:
credit_card_agg

,SK_ID_CURR,credit_card_MONTHS_BALANCE_max,credit_card_MONTHS_BALANCE_mean,credit_card_MONTHS_BALANCE_sum,credit_card_MONTHS_BALANCE_median,credit_card_MONTHS_BALANCE_std,credit_card_AMT_BALANCE_max,credit_card_AMT_BALANCE_mean,credit_card_AMT_BALANCE_sum,credit_card_AMT_BALANCE_median,credit_card_AMT_BALANCE_std,credit_card_AMT_CREDIT_LIMIT_ACTUAL_max,credit_card_AMT_CREDIT_LIMIT_ACTUAL_mean,credit_card_AMT_CREDIT_LIMIT_ACTUAL_sum,credit_card_AMT_CREDIT_LIMIT_ACTUAL_median,credit_card_AMT_CREDIT_LIMIT_ACTUAL_std,credit_card_AMT_DRAWINGS_ATM_CURRENT_max,credit_card_AMT_DRAWINGS_ATM_CURRENT_mean,credit_card_AMT_DRAWINGS_ATM_CURRENT_sum,credit_card_AMT_DRAWINGS_ATM_CURRENT_median,credit_card_AMT_DRAWINGS_ATM_CURRENT_std,credit_card_AMT_DRAWINGS_CURRENT_max,credit_card_AMT_DRAWINGS_CURRENT_mean,credit_card_AMT_DRAWINGS_CURRENT_sum,credit_card_AMT_DRAWINGS_CURRENT_median,credit_card_AMT_DRAWINGS_CURRENT_std,credit_card_AMT_DRAWINGS_OTHER_CURRENT_max,credit_card_AMT_DRAWINGS_OTHER_CURRENT_mean,credit_card_AMT_DRAWINGS_OTHER_CURRENT_sum,credit_card_AMT_DRAWINGS_OTHER_CURRENT_median,credit_card_AMT_DRAWINGS_OTHER_CURRENT_std,credit_card_AMT_DRAWINGS_POS_CURRENT_max,credit_card_AMT_DRAWINGS_POS_CURRENT_mean,credit_card_AMT_DRAWINGS_POS_CURRENT_sum,credit_card_AMT_DRAWINGS_POS_CURRENT_median,credit_card_AMT_DRAWINGS_POS_CURRENT_std,credit_card_AMT_INST_MIN_REGULARITY_max,credit_card_AMT_INST_MIN_REGULARITY_mean,credit_card_AMT_INST_MIN_REGULARITY_sum,credit_card_AMT_INST_MIN_REGULARITY_median,credit_card_AMT_INST_MIN_REGULARITY_std,credit_card_AMT_PAYMENT_CURRENT_max,credit_card_AMT_PAYMENT_CURRENT_mean,credit_card_AMT_PAYMENT_CURRENT_sum,credit_card_AMT_PAYMENT_CURRENT_median,credit_card_AMT_PAYMENT_CURRENT_std,credit_card_AMT_PAYMENT_TOTAL_CURRENT_max,credit_card_AMT_PAYMENT_TOTAL_CURRENT_mean,credit_card_AMT_PAYMENT_TOTAL_CURRENT_sum,credit_card_AMT_PAYMENT_TOTAL_CURRENT_median,credit_card_AMT_PAYMENT_TOTAL_CURRENT_std,credit_card_AMT_RECEIVABLE_PRINCIPAL_max,credit_card_AMT_RECEIVABLE_PRINCIPAL_mean,credit_card_AMT_RECEIVABLE_PRINCIPAL_sum,credit_card_AMT_RECEIVABLE_PRINCIPAL_median,credit_card_AMT_RECEIVABLE_PRINCIPAL_std,credit_card_AMT_RECIVABLE_max,credit_card_AMT_RECIVABLE_mean,credit_card_AMT_RECIVABLE_sum,credit_card_AMT_RECIVABLE_median,credit_card_AMT_RECIVABLE_std,credit_card_AMT_TOTAL_RECEIVABLE_max,credit_card_AMT_TOTAL_RECEIVABLE_mean,credit_card_AMT_TOTAL_RECEIVABLE_sum,credit_card_AMT_TOTAL_RECEIVABLE_median,credit_card_AMT_TOTAL_RECEIVABLE_std,credit_card_CNT_DRAWINGS_ATM_CURRENT_max,credit_card_CNT_DRAWINGS_ATM_CURRENT_mean,credit_card_CNT_DRAWINGS_ATM_CURRENT_sum,credit_card_CNT_DRAWINGS_ATM_CURRENT_median,credit_card_CNT_DRAWINGS_ATM_CURRENT_std,credit_card_CNT_DRAWINGS_CURRENT_max,credit_card_CNT_DRAWINGS_CURRENT_mean,credit_card_CNT_DRAWINGS_CURRENT_sum,credit_card_CNT_DRAWINGS_CURRENT_median,credit_card_CNT_DRAWINGS_CURRENT_std,credit_card_CNT_DRAWINGS_OTHER_CURRENT_max,credit_card_CNT_DRAWINGS_OTHER_CURRENT_mean,credit_card_CNT_DRAWINGS_OTHER_CURRENT_sum,credit_card_CNT_DRAWINGS_OTHER_CURRENT_median,credit_card_CNT_DRAWINGS_OTHER_CURRENT_std,credit_card_CNT_DRAWINGS_POS_CURRENT_max,credit_card_CNT_DRAWINGS_POS_CURRENT_mean,credit_card_CNT_DRAWINGS_POS_CURRENT_sum,credit_card_CNT_DRAWINGS_POS_CURRENT_median,credit_card_CNT_DRAWINGS_POS_CURRENT_std,credit_card_CNT_INSTALMENT_MATURE_CUM_max,credit_card_CNT_INSTALMENT_MATURE_CUM_mean,credit_card_CNT_INSTALMENT_MATURE_CUM_sum,credit_card_CNT_INSTALMENT_MATURE_CUM_median,credit_card_CNT_INSTALMENT_MATURE_CUM_std,credit_card_SK_DPD_max,credit_card_SK_DPD_mean,credit_card_SK_DPD_sum,credit_card_SK_DPD_median,credit_card_SK_DPD_std,credit_card_SK_DPD_DEF_max,credit_card_SK_DPD_DEF_mean,credit_card_SK_DPD_DEF_sum,credit_card_SK_DPD_DEF_median,credit_card_SK_DPD_DEF_std,credit_card_PNT_BALANCE_DRAWINGS_ATM_max,credit_card_PNT_BALANCE_DRAWINGS_ATM_mean,credit_card_PNT_BALANCE_DRAWINGS_ATM_sum,credit_card_PNT_BALANCE_DRAWINGS_ATM_median,credit_card_PNT_BALANCE_DRAWINGS_AT

In [40]:
len(credit_card_agg)

103558

Previous Application

In [41]:
raw_previous_application = pd.read_csv(os.getcwd()+"\\previous_application.csv")

In [42]:
raw_previous_application.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')

Previous Application Data Analysis

In [43]:
show_missing_values(raw_previous_application)

RATE_INTEREST_PRIVILEGED       99.64%
RATE_INTEREST_PRIMARY          99.64%
RATE_DOWN_PAYMENT              53.64%
AMT_DOWN_PAYMENT               53.64%
NAME_TYPE_SUITE                49.12%
NFLAG_INSURED_ON_APPROVAL       40.3%
DAYS_FIRST_DRAWING              40.3%
DAYS_FIRST_DUE                  40.3%
DAYS_LAST_DUE_1ST_VERSION       40.3%
DAYS_LAST_DUE                   40.3%
DAYS_TERMINATION                40.3%
AMT_GOODS_PRICE                23.08%
AMT_ANNUITY                    22.29%
CNT_PAYMENT                    22.29%
PRODUCT_COMBINATION             0.02%
CHANNEL_TYPE                     0.0%
NAME_PRODUCT_TYPE                0.0%
NAME_YIELD_GROUP                 0.0%
SELLERPLACE_AREA                 0.0%
NAME_SELLER_INDUSTRY             0.0%
NAME_GOODS_CATEGORY              0.0%
NAME_PORTFOLIO                   0.0%
SK_ID_PREV                       0.0%
NAME_CLIENT_TYPE                 0.0%
CODE_REJECT_REASON               0.0%
SK_ID_CURR                       0.0%
DAYS_DECISIO

In [44]:
show_cat_col_values(raw_previous_application)

                 NAME_CONTRACT_TYPE     Ratio
Cash loans                   747553  0.447579
Consumer loans               729151  0.436561
Revolving loans              193164  0.115652
XNA                             346  0.000207


           WEEKDAY_APPR_PROCESS_START     Ratio
TUESDAY                        255118  0.152746
WEDNESDAY                      255010  0.152681
MONDAY                         253557  0.151811
FRIDAY                         252048  0.150908
THURSDAY                       249099  0.149142
SATURDAY                       240631  0.144072
SUNDAY                         164751  0.098641


   FLAG_LAST_APPL_PER_CONTRACT     Ratio
Y                      1661739  0.994926
N                         8475  0.005074


                                  NAME_CASH_LOAN_PURPOSE     Ratio
XAP                                               922661  0.552421
XNA                                               677918  0.405887
Repairs                                            23765

In [45]:
raw_previous_application_dropped = raw_previous_application.drop(['SK_ID_PREV', 'HOUR_APPR_PROCESS_START','FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY','RATE_INTEREST_PRIVILEGED', 'RATE_INTEREST_PRIMARY'], axis = 1)

In [46]:
raw_previous_application_dropped.dtypes

SK_ID_CURR                      int64
NAME_CONTRACT_TYPE             object
AMT_ANNUITY                   float64
AMT_APPLICATION               float64
AMT_CREDIT                    float64
AMT_DOWN_PAYMENT              float64
AMT_GOODS_PRICE               float64
WEEKDAY_APPR_PROCESS_START     object
RATE_DOWN_PAYMENT             float64
NAME_CASH_LOAN_PURPOSE         object
NAME_CONTRACT_STATUS           object
DAYS_DECISION                   int64
NAME_PAYMENT_TYPE              object
CODE_REJECT_REASON             object
NAME_TYPE_SUITE                object
NAME_CLIENT_TYPE               object
NAME_GOODS_CATEGORY            object
NAME_PORTFOLIO                 object
NAME_PRODUCT_TYPE              object
CHANNEL_TYPE                   object
SELLERPLACE_AREA                int64
NAME_SELLER_INDUSTRY           object
CNT_PAYMENT                   float64
NAME_YIELD_GROUP               object
PRODUCT_COMBINATION            object
DAYS_FIRST_DRAWING            float64
DAYS_FIRST_D

Previous Application Feature Engineering

In [47]:
raw_previous_application_dropped['DAYS_DECISION'].replace(365243, np.nan, inplace= True)
raw_previous_application_dropped['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
raw_previous_application_dropped['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
raw_previous_application_dropped['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
raw_previous_application_dropped['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
raw_previous_application_dropped['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

In [48]:
raw_previous_application_dropped['DOWN_PAYMENT_PNT'] = raw_previous_application_dropped['AMT_DOWN_PAYMENT']/raw_previous_application_dropped['AMT_CREDIT']
raw_previous_application_dropped['ANNUITY_CREDIT_RATIO'] = raw_previous_application_dropped['AMT_ANNUITY']/raw_previous_application_dropped['AMT_CREDIT']

In [49]:
raw_previous_application_cats = pd.get_dummies(raw_previous_application_dropped)

In [50]:
raw_previous_application_dropped.dtypes

SK_ID_CURR                      int64
NAME_CONTRACT_TYPE             object
AMT_ANNUITY                   float64
AMT_APPLICATION               float64
AMT_CREDIT                    float64
AMT_DOWN_PAYMENT              float64
AMT_GOODS_PRICE               float64
WEEKDAY_APPR_PROCESS_START     object
RATE_DOWN_PAYMENT             float64
NAME_CASH_LOAN_PURPOSE         object
NAME_CONTRACT_STATUS           object
DAYS_DECISION                   int64
NAME_PAYMENT_TYPE              object
CODE_REJECT_REASON             object
NAME_TYPE_SUITE                object
NAME_CLIENT_TYPE               object
NAME_GOODS_CATEGORY            object
NAME_PORTFOLIO                 object
NAME_PRODUCT_TYPE              object
CHANNEL_TYPE                   object
SELLERPLACE_AREA                int64
NAME_SELLER_INDUSTRY           object
CNT_PAYMENT                   float64
NAME_YIELD_GROUP               object
PRODUCT_COMBINATION            object
DAYS_FIRST_DRAWING            float64
DAYS_FIRST_D

Previous Application Data Aggregation

In [51]:
raw_previous_application_agg_num = {
"AMT_ANNUITY":['max', 'mean','min','std'],
"AMT_APPLICATION":['max', 'mean','min','std'],
"AMT_CREDIT":['max', 'mean','min','std'],
"AMT_DOWN_PAYMENT":['max', 'mean','min','std'],
"AMT_GOODS_PRICE":['max', 'mean','min','std'],
"RATE_DOWN_PAYMENT":['max', 'mean','min','std'],
"DAYS_DECISION":['max', 'mean','min','std'],
"CNT_PAYMENT":['max', 'mean','min','std'],
"DAYS_FIRST_DRAWING":['max', 'mean','min','std'],
"DAYS_FIRST_DUE":['max', 'mean','min','std'],
"DAYS_LAST_DUE_1ST_VERSION":['max', 'mean','min','std'],
"DAYS_LAST_DUE":['max', 'mean','min','std'],
"DAYS_TERMINATION":['max', 'mean','min','std'],
"DOWN_PAYMENT_PNT":['max', 'mean','min','std', 'var'],
"ANNUITY_CREDIT_RATIO":['max', 'mean','min','std', 'var'],
}

raw_previous_application_cat_cols = [
    "NAME_CASH_LOAN_PURPOSE", 
    "NAME_PAYMENT_TYPE",
    "WEEKDAY_APPR_PROCESS_START",
    "CODE_REJECT_REASON",
    "NAME_CLIENT_TYPE",
    "NAME_GOODS_CATEGORY",
    "NAME_PORTFOLIO",
    "NAME_PRODUCT_TYPE",
    "CHANNEL_TYPE",
    "SELLERPLACE_AREA",
    "NAME_SELLER_INDUSTRY",
    "NAME_YIELD_GROUP",
]

for col in raw_previous_application_cat_cols:
    dict_ = get_cat_agg(raw_previous_application_cats, col, 'sum')
    raw_previous_application_agg_num.update(dict_)

raw_previous_application_agg_num

{'AMT_ANNUITY': ['max', 'mean', 'min', 'std'],
 'AMT_APPLICATION': ['max', 'mean', 'min', 'std'],
 'AMT_CREDIT': ['max', 'mean', 'min', 'std'],
 'AMT_DOWN_PAYMENT': ['max', 'mean', 'min', 'std'],
 'AMT_GOODS_PRICE': ['max', 'mean', 'min', 'std'],
 'RATE_DOWN_PAYMENT': ['max', 'mean', 'min', 'std'],
 'DAYS_DECISION': ['max', 'mean', 'min', 'std'],
 'CNT_PAYMENT': ['max', 'mean', 'min', 'std'],
 'DAYS_FIRST_DRAWING': ['max', 'mean', 'min', 'std'],
 'DAYS_FIRST_DUE': ['max', 'mean', 'min', 'std'],
 'DAYS_LAST_DUE_1ST_VERSION': ['max', 'mean', 'min', 'std'],
 'DAYS_LAST_DUE': ['max', 'mean', 'min', 'std'],
 'DAYS_TERMINATION': ['max', 'mean', 'min', 'std'],
 'DOWN_PAYMENT_PNT': ['max', 'mean', 'min', 'std', 'var'],
 'ANNUITY_CREDIT_RATIO': ['max', 'mean', 'min', 'std', 'var'],
 'NAME_CASH_LOAN_PURPOSE_Building a house or an annex': 'sum',
 'NAME_CASH_LOAN_PURPOSE_Business development': 'sum',
 'NAME_CASH_LOAN_PURPOSE_Buying a garage': 'sum',
 'NAME_CASH_LOAN_PURPOSE_Buying a holiday home /

In [52]:
previous_application_agg = raw_previous_application_cats.groupby('SK_ID_CURR').agg(raw_previous_application_agg_num)

previous_application_agg.columns = pd.Index(['prev_app_' + col[0] + "_" + col[1] for col in previous_application_agg.columns.tolist() if col[0] != 'SK_ID_CURR'])

previous_application_agg = previous_application_agg.reset_index()

In [53]:
len(previous_application_agg)

338857

Installment Payments

In [54]:
raw_installment_payments = pd.read_csv(os.getcwd()+"\\installments_payments.csv")

Installment Payments Data Analysis

In [55]:
show_missing_values(raw_installment_payments)

DAYS_ENTRY_PAYMENT        0.02%
AMT_PAYMENT               0.02%
SK_ID_PREV                 0.0%
SK_ID_CURR                 0.0%
NUM_INSTALMENT_VERSION     0.0%
NUM_INSTALMENT_NUMBER      0.0%
DAYS_INSTALMENT            0.0%
AMT_INSTALMENT             0.0%
dtype: object


In [56]:
raw_installment_payments.dtypes

SK_ID_PREV                  int64
SK_ID_CURR                  int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER       int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtype: object

In [57]:
raw_installment_payments_dropped = raw_installment_payments.drop(['SK_ID_PREV', 'NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER'], axis = 1)

Installment Payments Feature Engineering

In [58]:
raw_installment_payments_dropped['DAYS_INSTALMENT'].replace(365243, np.nan, inplace= True)
raw_installment_payments_dropped['DAYS_ENTRY_PAYMENT'].replace(365243, np.nan, inplace= True)

In [59]:
raw_installment_payments_agg_num = {
"DAYS_INSTALMENT": "mean",
"DAYS_ENTRY_PAYMENT": "mean",
"AMT_INSTALMENT": "mean",
"AMT_PAYMENT": "mean",
}

In [60]:
installment_payments_agg = raw_installment_payments_dropped.groupby('SK_ID_CURR').agg(raw_installment_payments_agg_num)

installment_payments_agg.columns = pd.Index(['install_payments_' + col + "_" + raw_installment_payments_agg_num[col] for col in installment_payments_agg.columns.tolist() if col != 'SK_ID_CURR'])

installment_payments_agg = installment_payments_agg.reset_index()

In [61]:
installment_payments_agg

,SK_ID_CURR,install_payments_DAYS_INSTALMENT_mean,install_payments_DAYS_ENTRY_PAYMENT_mean,install_payments_AMT_INSTALMENT_mean,install_payments_AMT_PAYMENT_mean
0,100001,-2187.714286,-2195.000000,5885.132143,5885.132143
1,100002,-295.000000,-315.421053,11559.247105,11559.247105
2,100003,-1378.160000,-1385.320000,64754.586000,64754.586000
3,100004,-754.000000,-761.666667,7096.155000,7096.155000
4,100005,-586.000000,-609.555556,6240.205000,6240.205000
...,...,...,...,...,...
339582,456251,-120.000000,-156.285714,7492.924286,7492.924286
339583,456252,-2391.000000,-2393.833333,10069.867500,10069.867500
339584,456253,-2372.928571,-2387.428571,4399.707857,4115.915357
339585,456254,-142.263158,-161.263158,10239.832895,10239.832895


In [62]:
len(installment_payments_agg)

339587

train_application

In [63]:
raw_train = pd.read_csv(os.getcwd()+"\\application_train.csv")

Data Analysis for the aggregated dataset will be performed in the main Exploratory Data Analysis File

In [64]:
print("application_train: ", len(raw_train))
print("bureau: ",len(bureau_agg))
print("credit_card_balance: ",len(credit_card_agg))
print("POS_CASH_balance: ",len(cash_balance_agg))
print("previous_application: ",len(previous_application_agg))
print("installments_payments: ",len(installment_payments_agg))


application_train:  307511
bureau:  305811
credit_card_balance:  103558
POS_CASH_balance:  337252
previous_application:  338857
installments_payments:  339587


In [65]:
train_agg = raw_train.merge(bureau_agg, how = 'left', on='SK_ID_CURR')

train_agg = train_agg.merge(credit_card_agg, how = 'left', on='SK_ID_CURR')

train_agg = train_agg.merge(cash_balance_agg, how = 'left', on='SK_ID_CURR')

train_agg = train_agg.merge(previous_application_agg, how = 'left', on='SK_ID_CURR')

train_agg = train_agg.merge(installment_payments_agg, how = 'left', on='SK_ID_CURR')

In [66]:
%cd home-credit-default-risk

raw_test = pd.read_csv(os.getcwd()+"\\application_test.csv")

test_agg = raw_test.merge(bureau_agg, how = 'left', on='SK_ID_CURR')

test_agg = test_agg.merge(credit_card_agg, how = 'left', on='SK_ID_CURR')

test_agg = test_agg.merge(cash_balance_agg, how = 'left', on='SK_ID_CURR')

test_agg = test_agg.merge(previous_application_agg, how = 'left', on='SK_ID_CURR')

test_agg = test_agg.merge(installment_payments_agg, how = 'left', on='SK_ID_CURR')

test_agg.to_csv("test_agg.csv", index = False)

[WinError 2] The system cannot find the file specified: 'home-credit-default-risk'
d:\Documents\Github\projs\home-credit-default-risk\data


In [67]:
print(len(train_agg))
print(len(train_agg.columns))
train_agg.head()

307511
473


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,bureau_DAYS_CREDIT_mean,bureau_CREDIT_DAY_OVERDUE_mean,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_max,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_CNT_CREDIT_PROLONG_max,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_var,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_LIMIT_max,bureau_AMT_CREDIT_SUM_OVERDUE_max,bureau_DAYS_CREDIT_UPDATE_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_var,bureau_CREDIT_TYPE_Another type of loan_sum,bureau_CREDIT_TYPE_Car loan_sum,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_sum,bureau_CREDIT_TYPE_Consumer credit_sum,bureau_CREDIT_TYPE_Credit card_sum,bureau_CREDIT_TYPE_Interbank credit_sum,bureau_CREDIT_TYPE_Loan for business development_sum,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,bureau_CREDIT_TYPE_Loan for the purchase of equipment_sum,bureau_CREDIT_TYPE_Loan for working capital replenishment_sum,bureau_CREDIT_TYPE_Microloan_sum,bureau_CREDIT_TYPE_Mobile operator loan_sum,bureau_CREDIT_TYPE_Mortgage_sum,bureau_CREDIT_TYPE_Real estate loan_sum,bureau_CREDIT_TYPE_Unknown type of loan_sum,credit_card_MONTHS_BALANCE_max,credit_card_MONTHS_BALANCE_mean,credit_card_MONTHS_BALANCE_sum,credit_card_MONTHS_BALANCE_median,credit_card_MONTHS_BALANCE_std,credit_card_AMT_BALANCE_max,credit_card_AMT_BALANCE_mean,credit_card_AMT_BALANCE_sum,credit_card_AMT_BALANCE_median,credit_card_AMT_BALANCE_std,credit_card_AMT_CREDIT_LIMIT_ACTUAL_max,credit_card_AMT_CREDIT_LIMIT_ACTUAL_mean,credit_card_AMT_CREDIT_LIMIT_ACTUAL_sum,credit_card_AMT_CREDIT_LIMIT_ACTUAL_median,credit_card_AMT_CREDIT_LIMIT_ACTUAL_std,credit_card_AMT_DRAWINGS_ATM_CURRENT_max,credit_card_AMT_DRAWINGS_ATM_CURRENT_mean,credit_card_AMT_DRAWINGS_ATM_C

In [68]:
print(len(train_agg[train_agg['TARGET']==1]))
print(len(train_agg[train_agg['TARGET']==0]))

24825
282686


In [69]:
train_agg.to_csv("train_agg.csv", index = False)